# XGB Classifier with Filter-Based FS (Lexical + Content)

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_new_Bacud_unbalanced_lexical.csv")      # Loading the dataset

dataset.head()

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [2]:
dataset.head()

,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [3]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [4]:
dataset['url_type'].value_counts()

url_type
0    724778
1    380244
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [5]:
important_features_lexical = ['url_host_length',
                             'url_is_https',
                             'url_ip_in_domain',
                             'has_php_in_string',
                             'url_number_of_parameters',
                             'has_exe_in_string',
                             'url_has_port',
                             'url_is_digits_in_domain',
                             'url_path_length',
                             'url_num_question_mark', 
                             'url_query_length',
                             'url_string_entropy',
                             'url_num_periods',
                             'get_tld',
                             'url_scheme']

X_test_lexical = x_test[important_features_lexical]
X_train_lexical = x_train[important_features_lexical]

#### Hyper-parameter Optimization

In [6]:
# Define the objective function for Optuna
def objective_lexical(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_lexical, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_lexical = optuna.create_study(direction='minimize')
study_lexical.optimize(objective_lexical, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_lexical = study_lexical.best_params
best_error_lexical = study_lexical.best_value
print("Best Hyperparameters (33 Features): ", best_params_lexical)
print("Best Error (33 Features): ", best_error_lexical)

[I 2024-04-12 11:19:04,309] A new study created in memory with name: no-name-3b080545-d792-4eb9-a979-44bb2ec87183


[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:19:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[1]	validation-error:0.65653
[2]	validation-error:0.65653
[3]	validation-error:0.65653
[4]	validation-error:0.41272
[5]	validation-error:0.22503
[6]	validation-error:0.15957
[7]	validation-error:0.12730
[8]	validation-error:0.11845
[9]	validation-error:0.10818
[10]	validation-error:0.09904
[11]	validation-error:0.09473
[12]	validation-error:0.09196
[13]	validation-error:0.08952
[14]	validation-error:0.08784
[15]	validation-error:0.08600
[16]	validation-error:0.08499
[17]	validation-error:0.08372
[18]	validation-error:0.08284
[19]	validation-error:0.08224
[20]	validation-error:0.08189
[21]	validation-error:0.08185
[22]	validation-error:0.08147
[23]	validation-error:0.08102
[24]	validation-error:0.08090
[25]	validation-error:0.08064
[26]	validation-error:0.08035
[27]	validation-error:0.08010
[28]	validation-error:0.07968
[29]	validation-error:0.07946
[30]	validation-error:0.07938
[31]	validation-error:0.07930
[32]	validation-error:0.07921
[33]	validation-error:0.07910
[34]	validation-err

[I 2024-04-12 11:19:33,555] Trial 0 finished with value: 0.27483744534912347 and parameters: {'eta': 0.10324474506906009, 'max_depth': 8, 'subsample': 0.5016507030916705, 'colsample_bytree': 0.872314403743355, 'gamma': 7.7722135184974395, 'min_child_weight': 7.856480476828913, 'lambda': 0.6882927448141689, 'alpha': 6.775732085023167}. Best is trial 0 with value: 0.27483744534912347.


...

[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:35:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-12 11:35:53,130] Trial 98 pruned. Trial was pruned at iteration 1.


[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:35:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-12 11:35:53,722] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters (33 Features):  {'eta': 0.29512096451884356, 'max_depth': 9, 'subsample': 0.9147612308171029, 'colsample_bytree': 0.7487957374869516, 'gamma': 0.7608999333495271, 'min_child_weight': 1.5863398412512155, 'lambda': 9.790649520817226, 'alpha': 1.8349450423539126}
Best Error (33 Features):  0.26822392448007276


#### Model Training

In [7]:
from sklearn.model_selection import KFold
from xgboost import DMatrix, train

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

best_params_lexical['objective'] = 'binary:hinge'
best_params_lexical['eval_metric'] = 'error'

# Convert the data into DMatrix format
lexical_train = DMatrix(X_train_lexical, label=y_train)
lexical_valid = DMatrix(X_test_lexical, label=y_test)

# Train the Model
xgb_classifier_lexical = train(best_params_lexical, lexical_train, num_boost_round=3000)
y_pred_lexical = xgb_classifier_lexical.predict(lexical_valid)

print("Model training done.")

Model training done.


In [8]:
# Classification Report
print(classification_report(y_test, y_pred_lexical))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94    144867
           1       0.92      0.87      0.89     76138

    accuracy                           0.93    221005
   macro avg       0.92      0.91      0.92    221005
weighted avg       0.93      0.93      0.93    221005



#### Practical Evaluation

In [9]:
# Dumping the model
joblib.dump(xgb_classifier_lexical, 'xgb_filter_lexical.sav')

['xgb_filter_lexical.sav']

In [10]:
import lexical_generator_filter_lexical
import time

def xgb_predict_maliciousness(url):

    numerical_values = lexical_generator_filter_lexical.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_lexical.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

print("------------- Filter-Based (Lexical) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Filter-Based (Lexical) -------------
Trial 0
Malware
0.07654090004507452
Trial 1
Malware
0.02213980001397431
Trial 2
Malware
0.021958600031211972
Trial 3
Malware
0.022510700044222176
Trial 4
Malware
0.02698699990287423
Trial 5
Malware
0.022244099993258715
Trial 6
Malware
0.02563340007327497
Trial 7
Malware
0.024678300018422306
Trial 8
Malware
0.022854399983771145
Trial 9
Malware
0.0246482000220567
Trial 10
Malware
0.022475300007499754
Trial 11
Malware
0.022576299961656332
Trial 12
Malware
0.022208599955774844
Trial 13
Malware
0.022816099924966693
Trial 14
Malware
0.021490099956281483


#### Evaluation

In [11]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred, labels=xgb_classifier.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier.classes_)\ndisp.plot()\nplt.show()'

In [12]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"